This works, but slow (4.2s per race)

To do:
- Fix place term errors
- Automation at certain time?
- Try optimise so its quicker
- Define the whole function with input being todays urls, tomorrows urls or all urls
- Misprice function needs work
- Specify if a race is green, arb or misprice


TO DO:
- Fix mp definition
- If one bookie doesn't have place terms it fails
- Instead of having all bookies (spreadex) can we limit even further?
- Email to say wether a race is green, arb or mp
    - Further on, the green should say if its sky green or else green

In [ ]:
import re
import cfscrape
import requests
import os
from bs4 import BeautifulSoup
import cloudscraper
import pandas as pd
from numpy import loadtxt
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail
from datetime import datetime
from random import randrange
import numpy as np

In [ ]:
start_time = datetime.now()

book = ["B3", "SK", "PP", "WH", "EE", "FB", "VC", "CE", "UN", "SX", "FR", "BY", "OE", "SA", "SI", "QN", "WA", "LD"]

In [ ]:
# Input: url
# Output: Oddschecker table with horses and decimal prices
# Working perfectly

def getTable(url):

    book = ["B3", "SK", "PP", "WH", "EE", "FB", "VC", "CE", "UN", "SX", "FR", "BY", "OE", "SA", "SI", "QN", "WA", "LD"]

    ids = ["diff-row evTabRow bc"]

    scraper = cloudscraper.create_scraper(delay=1,browser={'custom': 'ScraperBot/1.0',})

    url = url

    req = scraper.get(url)
    soup = BeautifulSoup(req.content, "html.parser")

    data = soup.find("tbody", {"id": "t1"})
    all_horses = soup.find_all("tr", {"class": ids})
    num_horses = len(all_horses)

    running_horses = []
    i = 0
    possible_nrs = 10
    j = 0

    while i < num_horses:
        horse_name = all_horses[i]['data-bname']
        running_horses.append(horse_name)
        i+=1

    while j < possible_nrs:
        num = "nr"
        running_horses.append(num)
        j+=1

    # b365 = data.find_all("td", {"data-bk": "B3"})

    # b365_prices = []
    # l = 0 

    # while l < len(b365):
    #    price = b365[l]['data-odig']
    #    b365_prices.append(price)
    #    l+=1

    df = pd.DataFrame()

    df['horse'] = running_horses

    gem = 0
    
    try:
        
        while gem < len(book):
            bookie = data.find_all("td", {"data-bk": book[gem]})

            prices = []
            l = 0

            while l < len(bookie):
                price = bookie[l]['data-odig']
                prices.append(price)
                l+=1

            df[book[gem]] = pd.Series(prices)
            gem+=1

        df = df[df.horse != 'nr']
    except:
        pass

    return df

In [ ]:
# Input: url (oddschecker table from getTable())
# Output: Boolean (T/F) if any win overround is less than 100%
# Working okay, not completely tested

def checkArb(df):
    book = ["B3", "SK", "PP", "WH", "EE", "FB", "VC", "CE", "UN", "SX", "FR", "BY", "OE", "SA", "SI", "QN", "WA", "LD"]
    df = getTable(url)
    all_winor = []
    isarb = 0
    i = 1
    total_bookies = df.shape[1] - 1

    # Get win overrounds for each bookie
    while i <= total_bookies:

        df[book] = df[book].apply(pd.to_numeric)

        individual_book = df.iloc[:,i]

        temp_df = pd.DataFrame(individual_book)

        temp_df['prob'] = 1/temp_df.iloc[:,0]

        win_or = round(temp_df['prob'].sum()*100,2)

        all_winor.append(win_or)

        i+=1

        
    # Check if any element in the win overrounds is less than 100%
    isarb = all(i < 100 for i in all_winor)

    return isarb

In [ ]:
# Input: An individual bookies prices (decimal), place terms (usually 4 or 5), places_paying (any number from 1 to 8)
# Output: Boolean (T/F) if any place overround is less than the places_paying (i.e green book)
# Working okay, not fully tested

def getPlaceor(individual_book, place_term, places_paying):
    
    temp_df = pd.DataFrame(individual_book)
    temp_df['minus1'] = temp_df.iloc[:,0] - 1
    temp_df['dividebyterm'] = temp_df['minus1'] / place_term
    temp_df['dividebyterm'] = temp_df['dividebyterm'] + 1
    temp_df['placeor'] = 1/temp_df['dividebyterm']

    o_r = round(temp_df['placeor'].sum(),2)
    
    if o_r < places_paying:
        isgreen = True
    else:
        isgreen = False
    
    return isgreen

In [ ]:
# Input: URL
# Output: Place terms and places paying for each bookie
# Working okay, not fully tested

def getEWTerms(url):
    
    book = ["B3", "SK", "PP", "WH", "EE", "FB", "VC", "CE", "UN", "SX", "FR", "BY", "OE", "SA", "SI", "QN", "WA", "LD"]

    ids = ["diff-row evTabRow bc"]

    scraper = cloudscraper.create_scraper(delay=10,browser={'custom': 'ScraperBot/1.0',})

    url = url

    req = scraper.get(url)
    soup = BeautifulSoup(req.content, "html.parser")

    data = soup.find("tbody", {"id": "t1"})
    all_horses = soup.find_all("tr", {"class": ids})
    num_horses = len(all_horses)
    
    running_horses = []
    i = 0
    possible_nrs = 10
    j = 0

    while i < num_horses:
        horse_name = all_horses[i]['data-bname']
        running_horses.append(horse_name)
        i+=1

    while j < possible_nrs:
        num = "nr"
        running_horses.append(num)
        j+=1

    # b365 = data.find_all("td", {"data-bk": "B3"})


    # place_term = b365[0]['data-ew-denom']
    # places_paying = b365[0]['data-ew-places']


    df = pd.DataFrame()

    df['horse'] = running_horses

    gem = 0

    place_terms = []
    places_paying = []
    
    try:
        while gem < len(book):
            bookie = data.find_all("td", {"data-bk": book[gem]})

            place_term = bookie[0]['data-ew-denom']
            place_paying = bookie[0]['data-ew-places']

            place_term = int(place_term)
            place_paying = int(place_paying)

            place_terms.append(place_term)
            places_paying.append(place_paying)

            gem+=1

        dfnew = pd.DataFrame(columns=[book])

        dfnew.loc[0] = place_terms
        dfnew.loc[1] = places_paying

        dfnew.rename(index={0: 'PlaceTerms', 1: 'PlacesPaying'}, inplace=True)
    
        return dfnew
    
    except:
        print("Error: place terms not found -", url)

In [ ]:
# Input: URL
# Output: Boolean (T/F) whether any bookie offers green E/W


def checkGreen(url):
    
    book = ["B3", "SK", "PP", "WH", "EE", "FB", "VC", "CE", "UN", "SX", "FR", "BY", "OE", "SA", "SI", "QN", "WA", "LD"]

    dfew = getEWTerms(url)

    df = getTable(url)

    df = df.drop(columns='horse')

    total_bookies = df.shape[1]

    i = 0

    green_list = []
    
    try:

        while i < total_bookies:

            df[book] = df[book].apply(pd.to_numeric)
            individual_book = df.iloc[:,i]

            placeterm = dfew.iloc[0,i]
            placespaying = dfew.iloc[1,i]

            colour = getPlaceor(individual_book, placeterm, placespaying)

            green_list.append(colour)

            i+=1

        isgreen = any(i == True for i in green_list)

        return isgreen
    
    except:
        pass

In [ ]:
def filterLinks(string, substr):
    return [str for str in string if
            any(sub in str for sub in substr)]

In [ ]:
def getLinks():
    
    racecourses = pd.read_csv('racecourses.txt', sep=",", header=None)[0].tolist() 
    
    links = []

    scraper = cloudscraper.create_scraper(delay=10,browser={'custom': 'ScraperBot/1.0',})

    url = "https://www.oddschecker.com/horse-racing/"

    page = scraper.get(url)    
    data = page.text
    soup = BeautifulSoup(data)

    for link in soup.find_all('a'):
        links.append(link.get('href'))

    links1 = []
    for val in links:
        if val != None :
            links1.append(val)

    links2 = [k for k in links1 if 'winner' in k]

    links3 = [k for k in links2 if '/horse-racing/' in k]

    links4 = links3[:140]
    
    urls = links4
    
    urls = [f"https://www.oddschecker.com{l}"for l in urls]
    
    urls = filterLinks(urls, racecourses)
    
    return urls

In [ ]:
def numpy_triu1(df):          
    a = df.values
    r,c = np.triu_indices(a.shape[1],1)
    cols = df.columns
    nm = [cols[i]+"_"+cols[j] for i,j in zip(r,c)]
    return pd.DataFrame(a[:,r] - a[:,c], columns=nm)

In [ ]:
def checkMP(url, MP_THRESHOLD):

    MP_THRESHOLD = 0.08 # Set this to whatever you want (0.07 = 7%)

    url = url

    book = ["B3", "SK", "PP", "WH", "EE", "FB", "VC", "CE", "UN", "SX", "FR", "BY", "OE", "SA", "SI", "QN", "WA", "LD"]

    df = getTable(url)
    df = df.drop(columns='horse')
    total_bookies = df.shape[1]

    df = df.astype(float)

    df_percent = df.pow(-1).round(2)


    df_percent.replace([np.inf, -np.inf], 0.00, inplace=True)

    all_ov = list(df_percent.iloc[0:].sum())
    books_open_asper = sum(x > 1 for x in all_ov) / len(all_ov)
    b365 = all_ov[0]
    pp = all_ov[2]

    if books_open_asper > 0.5:
        df_diff = numpy_triu1(df_percent)

        df_diff = df_diff.abs()

        a = list(df_diff.max())

        max_diff = max(a)

        ismp = max_diff > MP_THRESHOLD
    else:
        ismp = False

        return ismp

In [ ]:
years = ['2022','2023','2024']

urls = getLinks()

urls_tomorrow = filterLinks(urls, years)

urls_today = [x for x in urls if x not in urls_tomorrow]

races_to_check = []

In [ ]:
# This filters out races that have already taken place

now = datetime.now()
current_time = datetime.now()
current_day = datetime.today().strftime('%Y-%m-%d')

i=0
urls_today_clean = []

while i < len(urls_today):

    time = re.search('/horse-racing/(.*)/winner', urls_today[i])
    time = time.group(1)
    time = time[-5:]
    time = now.replace(hour=int(time[:2]), minute=int(time[-2:]))

    if current_time < time:
        urls_today_clean.append(urls_today[i])
        i+=1
    else:
        i+=1

In [ ]:
# Adding cleaned today's urls to tomorrows to get all urls (clean)

all_urls_clean = urls_today_clean + urls_tomorrow

In [ ]:
# this is the main driver of the project

MP_THRESHOLD = 0.07 # Set this to whatever you want (0.07 = 7%)
links_to_use = all_urls_clean # change this by desired races to scrape (all_urls_clean, urls_today_clean or urls_tomorrow)

i = 0

while i < len(links_to_use):
    url = links_to_use[i]

    arbcheck = checkArb(url)
    greencheck = checkGreen(url)
    #mpcheck = checkMP(url, MP_THRESHOLD)

    if arbcheck or greencheck:# or mpcheck:
        races_to_check.append(url)
    else:
        races_to_check
    print(i)
    i+=1

In [ ]:
all_to_send = []
k=0

while k < len(races_to_check):
    to_send = re.search('/horse-racing/(.*)/winner', races_to_check[k])
    to_send = to_send.group(1)
    to_send = to_send.replace('/',' ',1)
    all_to_send.append(to_send)
    k+=1

In [ ]:
all_to_send = '<br>'.join(all_to_send)

In [ ]:
# this calculates time to scrape 

end_time = datetime.now()

duration = end_time - start_time                         
duration = divmod(duration.total_seconds(),60)
mins = str(int(round(duration[0],0)))
seconds = str(int(round(duration[1],0)))

time_info = "time_to_scan: "+mins+" minutes "+seconds+" seconds"
races_scanned = "races_scanned: "+str(i)

In [ ]:
message_content= 'Hi,<br>The following races are worth looking at:<p><p>'+str(all_to_send)+'</p><p><p>Thanks,<br>1mikebot1</p><p></p>'+time_info+'</p><p>'+races_scanned+'<p>www.oddschecker.com/horse-racing<p>'

In [ ]:
email_list = ['michael98mullin@gmail.com']

In [ ]:
# Mailer
number = randrange(100)

message = Mail(
    from_email='1mikebot1@gmail.com',
    to_emails=email_list,
    subject="Oddschecker ("+current_day+") #"+str(number),
    html_content=message_content)
try:
    sg = SendGridAPIClient(APIKEY)
    response = sg.send(message)
    print(response.status_code)
    print(response.body)
    print(response.headers)
except Exception as e:
    print(e)

number+=1